In [20]:
import os

import pandas as pd
import numpy as np
import janitor
import loguru
from loguru import logger

#### Constantes pour constants.py

In [2]:
path_to_test = "/media/sacha/DATA_P21/projet_RLQ_Odoo_test/input_files/"
file_to_test =  path_to_test + "event_registration.xlsx"

### initialisation des logs dasn le dossier logs/

In [21]:
# Initialisation du fichier log dans le dossier /logs et ajout de logs colorés

log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, "notebook.log")


logger.remove()  # retire la configuration par défaut
logger.add(log_file, rotation="1 MB", retention="10 days", level="INFO", encoding="utf-8")

def log_colored(message, level="success"):
    if level == "success":
        logger.opt(colors=True).success(f"<green>{message}</green>")
    elif level == "warning":
        logger.opt(colors=True).warning(f"<yellow>{message}</yellow>")
    elif level == "error":
        logger.opt(colors=True).error(f"<red>{message}</red>")
    else:
        logger.info(message)

# Exemples d'utilisation :
log_colored("Opération réussie.", "success")
log_colored("Attention : vérifiez les données.", "warning")
log_colored("Erreur lors du traitement.", "error")

# Exemple d'utilisation après une clause except :
# except Exception as e:
#     logger.exception(f"Une erreur est survenue : {e}")
#     print(f"Une erreur est survenue : {e}")

#### Lecture du fichier et renommage du fichier en event_registration.xlsx

In [ ]:

# Nouveau nom souhaité pour le fichier.
new_filename = "event_registration.xlsx"

# --- Logique du script ---

print(f"Vérification du dossier '{path_to_test}'...")

# S'assurer que le dossier d'entrée existe, sinon le créer.
if not os.path.isdir(path_to_test):
    log_colored(f"Le dossier '{path_to_test}' n'a pas été trouvé. Création du dossier.", "warning")
    os.makedirs(path_to_test)
    log_colored(
        f"--> Veuillez maintenant placer un fichier dans le dossier '{path_to_test}' et ré-exécuter la cellule.",
        "warning"
    )
else:
    try:
        # Lister les fichiers dans le dossier (en excluant les sous-dossiers).
        files_in_dir = [
            f
            for f in os.listdir(path_to_test)
            if os.path.isfile(os.path.join(path_to_test, f))
        ]

        # Gérer les différents cas.
        if len(files_in_dir) == 0:
            log_colored(f"Aucun fichier trouvé dans le dossier '{path_to_test}'."), "error"
            log_colored(f"--> Veuillez placer un fichier dans le dossier '{path_to_test}' et ré-exécuter la cellule.", "error")

        elif len(files_in_dir) > 1:
            log_colored(
                f"ERREUR : Plusieurs fichiers trouvés dans '{path_to_test}'. Le script ne peut en traiter qu'un seul à la fois.",
                "error"
            )
            log_colored("Fichiers trouvés :", files_in_dir,"success")

        else:
            # Il y a exactement un fichier, on peut le renommer.
            original_filename = files_in_dir[0]

            # Construire les chemins complets (ancien et nouveau).
            old_filepath = os.path.join(path_to_test, original_filename)
            new_filepath = os.path.join(path_to_test, new_filename)

            # Vérifier si le fichier a déjà le bon nom.
            if old_filepath == new_filepath:
                log_colored(
                    f"Le fichier est déjà nommé '{new_filename}'. Aucune action nécessaire.",
                    "warning"
                )
            else:
                # Renommer le fichier.
                os.rename(old_filepath, new_filepath)
                log_colored(
                    f"SUCCÈS : Le fichier '{original_filename}' a été renommé en '{new_filename}'.",
                    "success"
                )

    except OSError as e:
        log_colored(f"Une erreur système est survenue : {e}","error")

#### lecture du fichier event-registration.xlsx

In [4]:
df_to_test = pd.read_excel(file_to_test)
df_to_test.head()

,Actif,Créé le,Nom du participant,Réservé par,Email,Téléphone,Nom de la société,Événement,Ticket d'événement,Activités,Réponses des participants,Status,Statut de la vente,Nécessite une action,Activité exception décoration
0,1.0,2025-09-18 15:51:10,Matuzalem Muller,Matuzalem Muller,matuzalem.santos@gmail.com,NaN,NaN,Journée portes ouvertes sur Linux 2025 (418 pl...,Visiteur (248 places restantes),NaN,Matuzalem Muller,Inscrit,Gratuit,0.0,NaN
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matuzalem Muller,NaN,NaN,NaN,NaN
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matuzalem.santos@gmail.com,NaN,NaN,NaN,NaN
3,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non,NaN,NaN,NaN,NaN
4,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oui,NaN,NaN,NaN,NaN


#### cleaning des noms de colonnes

In [5]:
liste_df_columns = df_to_test.columns.tolist()
log_colored(liste_df_columns, "success")

In [6]:
df_clean_to_test = df_to_test.clean_names().remove_empty()

In [7]:
liste_col_new_df = df_clean_to_test.columns.tolist()
log_colored(liste_col_new_df, "success")

#### Creation du fichier de sortie clean et du path de sortie

In [8]:
output_dir = "/media/sacha/DATA_P21/projet_RLQ_Odoo_test/ouput_files"
os.makedirs(output_dir, exist_ok=True)

# Définir le chemin complet du fichier de sortie
output_file_cleaned = os.path.join(output_dir, "event_registration_cleaned.xlsx")


df_clean_to_test.to_excel(output_file_cleaned, index=False)

log_colored(f"Fichier nettoyé sauvegardé sous : {output_file_cleaned}", "success")

#### Creation de la liste des type de participants

In [9]:
liste_values_ticket = [
    x for x in df_clean_to_test["ticket_devenement"].unique() if str(x) != "nan"
]
log_colored(liste_values_ticket, "success")

#### Cleaning colonne Status

In [10]:
df_clean_to_test['status'].unique()
df_clean_to_test['status'].fillna(np.nan)
df_clean_to_test['status'] = df_clean_to_test['status'].replace({np.nan: 'absent'})
df_clean_to_test["status"] = df_clean_to_test["status"].replace({'Présent': "present"})
df_clean_to_test["status"] = df_clean_to_test["status"].replace({"Inscrit": "inscrit"})
liste_status = [x for x in df_clean_to_test['status'].unique()]
log_colored(liste_status, "success")

#### Creation des dataframes sponsor, benevoles et visiteurs 

In [11]:
df_filter_sponsor = df_clean_to_test[df_clean_to_test['ticket_devenement'] == liste_values_ticket[1]]

In [12]:
df_sponsors = df_filter_sponsor[df_filter_sponsor['status'] == 'present']
log_colored(f"Nombre de sponsors présents : {len(df_sponsors)}", "success")

In [13]:
df_filter_benevoles = df_clean_to_test[
    df_clean_to_test["ticket_devenement"] == liste_values_ticket[2]
]

In [14]:
df_benevoles = df_filter_benevoles[df_filter_benevoles['status'] == 'present']
log_colored(f"Nombre de bénévoles présents : {len(df_benevoles)}", "success")

In [15]:
df_filter_visiteurs = df_clean_to_test[
    df_clean_to_test["ticket_devenement"] == liste_values_ticket[0]
]
len(df_filter_visiteurs)

52

In [16]:
df_visiteurs = df_filter_visiteurs[df_filter_visiteurs['status'] == 'present']
log_colored(f"Nombre de visiteurs présents : {len(df_visiteurs)}", "success")

#### Creation des fichiers benevoles, sponsors et visiteurs

In [17]:
output_file_sponsors = os.path.join(output_dir, "event_registration_sponsors.xlsx")
df_sponsors.to_excel(output_file_sponsors, index=False)
log_colored(f"Fichier sponsors sauvegardé sous : {output_file_sponsors}", "success")

In [18]:
output_file_benevoles = os.path.join(output_dir, "event_registration_benevoles.xlsx")
df_benevoles.to_excel(output_file_benevoles, index=False)
log_colored(f"Fichier bénévoles sauvegardé sous : {output_file_benevoles}", "success")

In [19]:
output_file_visiteurs = os.path.join(output_dir, "event_registration_visiteurs.xlsx")
df_visiteurs.to_excel(output_file_visiteurs, index=False)
log_colored(f"Fichier visiteurs sauvegardé sous : {output_file_visiteurs}", "success")